In [1]:
import pandas as pd
import requests
import psycopg2
import os
import sqlalchemy
import json
import numpy as np

In [2]:
# Database credentials
ENDPOINT = 'datalaketwitter.c16p7bc6xhi2.us-east-1.rds.amazonaws.com'
DB_NAME = 'datalakeTwitter'
USERNAME = 'nadine'
PASSWORD = '30101995'

In [3]:
# connecting to the database
try:
    print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
        
try:
    cur = conn.cursor()
    engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(USERNAME,PASSWORD,ENDPOINT,DB_NAME))

    
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
        
# Create sqlalchemy engine
engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(USERNAME,PASSWORD,ENDPOINT,DB_NAME))

host=datalaketwitter.c16p7bc6xhi2.us-east-1.rds.amazonaws.com dbname=datalakeTwitter user=nadine password=30101995


In [4]:
conn.set_session(autocommit=True)

In [5]:
#fetching data from datalake and save it as variable
data_history = engine.execute('Select * FROM tasty_twitter_history;').fetchall()

In [6]:
#creating a pandas dataframe for further steps
df_history = pd.DataFrame(data_history)

In [7]:
# creating new columns
df_history['url_short'] = ''
df_history['displayed_url'] = ''
df_history['expanded_url'] = ''
df_history['symbols'] = ''
#df['hashtags'] = ''
#df['user_mentions'] = ''

In [8]:
# fetch the link out of the additional_information and appends it to the newly created colums above
for i, row in df_history.iterrows(): 
    urls = (row.additional_information.get('urls'))
    #df['media'].iloc[i] = row.additional_information.get('media')
    df_history['symbols'].iloc[i] = row.additional_information.get('symbols')
    #df['hashtags'].iloc[i] = row.additional_information.get('hashtags')
    #df['user_mentions'].iloc[i] = row.additional_information.get('user_mentions')
    if not urls:
        df_history['url_short'].iloc[i] = np.nan
        df_history['displayed_url'].iloc[i] = np.nan
        df_history['expanded_url'].iloc[i] = np.nan
    else:
        for url in urls:
            df_history['url_short'].iloc[i]= url.get('url')
            df_history['displayed_url'].iloc[i]= url.get('display_url')
            df_history['expanded_url'].iloc[i]= url.get('expanded_url')

C:\Users\nadin\AppData\Local\Temp\ipykernel_14628\452509909.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_history['symbols'].iloc[i] = row.additional_information.get('symbols')
C:\Users\nadin\AppData\Local\Temp\ipykernel_14628\452509909.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_history['url_short'].iloc[i]= url.get('url')
C:\Users\nadin\AppData\Local\Temp\ipykernel_14628\452509909.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [10]:
# filling in empty values with na and dropping column additional_information
df_history = df_history.fillna(value=np.nan)
df_history = df_history.drop(['additional_information'], axis = 1)
df_history.symbols = df_history.symbols.apply(lambda y: np.nan if len(y)==0 else y)
#df.hashtags = df.hashtags.apply(lambda y: np.nan if len(y)==0 else y)
#df.user_mentions = df.user_mentions.apply(lambda y: np.nan if len(y)==0 else y)
#display(df_history)

KeyError: "['additional_information'] not found in axis"

In [11]:
# in the following steps the recipe name is extraxted out of the link
# first a column with 0 or 1 is created to indicate if the tweet has a recipe in it
recipe_link_list_history = []

for item in df_history['tweet']:
    if 'Shop the recipe!' in item:
        recipe_link_list_history.append(1)
    else:
        recipe_link_list_history.append(0)
df_history['recipe_link'] = recipe_link_list_history

In [12]:
# the type is adapted and a sub dataframe is created for the furhter steps
df_history['expanded_url'] = df_history['expanded_url'].astype(str)
df_history_recipes_yes = df_history[df_history['recipe_link'] == 1]

In [13]:
# the link is splited and joined to extract the recipe name which is then appended to a list
recipe_name_history = []
for recipe in df_history_recipes_yes['expanded_url']:
    test = recipe.split('/')
    if len(test) >= 4:
        test_split= test[4].split('-')
        test = " ".join(test_split)
        recipe_name_history.append(test)

In [14]:
# out of the recipe name an iterater is generated to append the recipe name to the rows that have a recipe
iter_recipe_name_history = iter(recipe_name_history)
df_history_recipe_name = []

for element in df_history['recipe_link']:
    if element == 1:
        df_history_recipe_name.append(next(iter_recipe_name_history))
    else:
        df_history_recipe_name.append(np.nan)

In [15]:
# adding recipe name to dataframe
df_history['recipe_name'] = df_history_recipe_name

In [17]:
#display(df_history)

,tweet,created,favorites,count,url_short,displayed_url,expanded_url,symbols,recipe_link,recipe_name
0,Betcha haven't tried em all. https://t.co/KN7V...,2022-04-01,40,7,https://t.co/KN7VyNQKfK,buzzfeed.com/kevinsmith/are…,https://www.buzzfeed.com/kevinsmith/are-you-a-...,NaN,0,NaN
1,Betcha haven't tried em all. https://t.co/KN7V...,2022-04-01,40,7,https://t.co/KN7VyNQKfK,buzzfeed.com/kevinsmith/are…,https://www.buzzfeed.com/kevinsmith/are-you-a-...,NaN,0,NaN
2,"Sweet babies, salty babies, or no babies? http...",2022-04-01,2,1,https://t.co/bBZchqWiWc,buzzfeed.com/perpetua/this-…,https://www.buzzfeed.com/perpetua/this-sweet-v...,NaN,0,NaN
3,Betcha haven't tried em all. https://t.co/KN7V...,2022-04-01,40,7,https://t.co/KN7VyNQKfK,buzzfeed.com/kevinsmith/are…,https://www.buzzfeed.com/kevinsmith/are-you-a-...,NaN,0,NaN
4,There's no 50:50. https://t.co/Q66POl0Lar,2022-04-01,58,6,https://t.co/Q66POl0Lar,buzzfeed.com/remeepatel/are…,https://www.buzzfeed.com/remeepatel/are-you-mo...,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...
3247,Country Fried Steak with thick gravy on top......,2021-12-27,205,47,https://t.co/pj3Ky7MozB,tasty.co/recipe/country…,https://tasty.co/recipe/country-fried-steak-an...,NaN,1,country fried steak and gravy
3248,Have you watched how empanadas are made. 😍 htt...,2021-12-27,447,86,NaN,NaN,nan,NaN,0,NaN
3249,Meal prep for the week and make life easier fo...,2021-12-27,223,50,https://t.co/6wZSeEnjyA,tasty.co/recipe/balsami…,https://tasty.co/recipe/balsamic-chicken-and-v...,NaN,1,balsamic chicken and veggies meal prep
3250,Save these recipes for when you plan your next...,2021-12-27,90,22,https://t.co/INcxLUYVzB,tasty.co/recipe/spicy-c…,https://tasty.co/recipe/spicy-campfire-queso,NaN,1,spicy campfire queso


In [16]:
#creating table if not exists
conn.set_session(autocommit=True)
cur.execute("CREATE TABLE IF NOT EXISTS tasty_twitter_history_clean (tweet TEXT, created DATE, favorites INT, count INT, url_short TEXT, displayed_url TEXT, expanded_url TEXT, symbols TEXT, recipe_link CHAR, recipe_name TEXT);")

In [18]:
#adding the df into the database
df_history.to_sql('tasty_twitter_history_clean', engine, if_exists = 'replace', index = False)

252

In [20]:
#close the connections
conn.close()
#engine.close()